In [1]:
from dotenv import load_dotenv , find_dotenv
import os
dotenv_path = find_dotenv()

load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("OPENAI_API_KEY not found in environment variables")


In [3]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator 
from langchain.embeddings import OpenAIEmbeddings # importing embedding models 
from langchain import OpenAI
import openai
from langchain_community.vectorstores import FAISS


In [4]:
#loading the PDF
loader  = PyPDFLoader("C:\\Users\\Admin\\Documents\\gen_ai_training\\pdfs\\rag.pdf")
docs = loader.load()

#splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, add_start_index=True)
chunks = text_splitter.split_documents(docs)


In [5]:
# Embedding 
#Using FAISS vector DB
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embedding=embeddings)  # Pass embeddings as parameter
print(f"Total chunks indexed: {db.index.ntotal}")

c:\Users\Admin\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Total chunks indexed: 276


In [6]:
#Integrating with openAI to get better query results
llm = OpenAI(temperature=0.2)

def generate_response(query, llm, db, top_k=4):
    # Perform similarity search to retrieve relevant text chunks
    chunks = db.similarity_search(query, k=top_k)
    
    # Combine the content of the top-k chunks into a single text string
    combined_text = "\n".join([chunk.page_content for chunk in chunks])
    
    

    # Generate response using OpenAI API
    response =openai.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"Based on the following content, answer the query and if its not in the content then say i dont know : {query}\n\nContent:\n{combined_text}\n\nAnswer:",
        max_tokens = 150
    )
     # Extract the response text
    response_text = response.choices[0].text.strip()
    
    # Return response along with metadata
    return response_text





c:\Users\Admin\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [11]:
query = "explain thoroughly the 3 types of RAG?"
response = generate_response(query, embeddings, db)
print(f"Response to '{query}': {response}")


Response to 'explain thouroughly the 3 types of RAG?': The three types of RAG are Naive RAG, Advanced RAG, and Modular RAG. Naive RAG mainly consists of three parts: indexing, retrieval, and generation. Advanced RAG proposes multiple optimization strategies and has a similar process to Naive RAG. Modular RAG showcases greater flexibility overall and responds to the specific shortcomings of Naive RAG. RAG's technical integration with other AI methodologies, such as fine-tuning and reinforcement learning, has further expanded its capabilities. Despite the progress in RAG technology, there are still opportunities for research to improve its robustness and ability to handle extended contexts. RAG's application scope is also expanding into multimodal domains.
